In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from keras import backend as K

import os
os.chdir("..")
from src import *


2023-03-25 12:55:27.325962: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-25 12:55:27.522327: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-25 12:55:27.522355: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-25 12:55:28.950499: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


## Load data

In [3]:
_df = pd.read_csv("Data/wig20_d.csv")
# df = pd.read_csv("Data/mwig40_d.csv")
# df = pd.read_csv("Data/swig80_d.csv")

_df.drop(range(1000), inplace=True)
_df.reset_index(inplace=True, drop=True)
_df.head()

,Data,Otwarcie,Najwyzszy,Najnizszy,Zamkniecie,Wolumen
0,1997-02-06,1717.2,1779.8,1717.2,1779.8,4435886.0
1,1997-02-07,1778.2,1808.7,1778.2,1799.0,5448243.0
2,1997-02-10,1797.4,1797.4,1780.3,1783.3,6513315.0
3,1997-02-11,1803.3,1832.2,1803.3,1832.0,5146340.0
4,1997-02-12,1824.6,1824.6,1804.9,1810.2,5748398.0


In [4]:
data = _df.drop(columns=["Data"]).values.transpose()
data.shape

(5, 6429)

## Process and split data

In [8]:
thr_fall, thr_rise = (-0.005270574305918364, 0.004968199728502841)

data_pipeline = DataProcess(
    data, test_ratio=0.2, validation_ratio=0.2, batch_size=32,
    threshold_fall=thr_fall, threshold_rise=thr_rise,
    feature_to_predict_num=3,
    balance_training_dataset=True
)
data_pipeline.run()
_x_train, _y_train, _x_validation, _y_validation, _x_test, _y_test = data_pipeline.get_data()
_y_test_ind, _y_validation_ind = np.argmax(_y_test, axis=1), np.argmax(_y_validation, axis=1)


In [9]:
pd.DataFrame(
    {
        "x": [_x_train.shape, _x_test.shape, _x_validation.shape],
        "y": [_y_train.shape, _y_test.shape, _y_validation.shape]
    },
    index = ["train", "test", "validation"]
)


,x,y
train,"(3336, 5, 32, 2)","(3336, 3)"
test,"(1254, 5, 32, 2)","(1254, 3)"
validation,"(1742, 5, 32, 2)","(1742, 3)"


In [10]:
print("Train dataset classes value counts:")
_y_train.sum(axis=0)
# pd.DataFrame(
#     {
#         str(key): val for (key, val) in zip(*np.unique(np.argmax(_y_train, axis=1), return_counts=True))
#     },
#     index = ["count"]
# )

Train dataset classes value counts:


array([1112, 1112, 1112])

## Model

### Building

In [8]:
model = tf.keras.Sequential()

model.add(tf.keras.Input(shape=_x_train.shape[1:]))
for i in range(7):
    model.add(tf.keras.layers.Conv2D(2, (1, 3), 1, padding="same"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(1, 3), strides=(1, 3), padding='same'))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(12))
model.add(tf.keras.layers.Dense(7))
model.add(tf.keras.layers.Dense(3, activation="softmax"))


2023-03-17 00:05:56.543123: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-17 00:05:56.543176: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-17 00:05:56.543200: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (banjaro): /proc/driver/nvidia/version does not exist
2023-03-17 00:05:56.543495: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
model.build(input_shape=_x_train.shape[1:])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 5, 32, 2)          14        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 5, 11, 2)         0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 5, 11, 2)          14        
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 4, 2)          0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 5, 4, 2)           14        
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 5, 2, 2)          0

In [11]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              # metrics=f1_m
              )


In [12]:
# es = tf.keras.callbacks.EarlyStopping(monitor='val_f1_m',
#                                       mode='max',
#                                       min_delta=1e-3,
#                                       patience=20,
#                                       start_from_epoch=200,
#                                       )
# mc = tf.keras.callbacks.ModelCheckpoint('Models/cdt_2/best_model.h5',
#                                         monitor='val_f1_m',
#                                         mode='max'
#                                         )

history = model.fit(_x_train, _y_train,
                    epochs=100,
                    # callbacks=[es, mc],
                    # validation_data=(_x_test, _y_test)
                    )


Epoch 1/100
 37/120 [========>.....................] - ETA: 0s - loss: 1.1080

2023-03-17 00:05:58.554802: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 21947040 exceeds 10% of free system memory.
2023-03-17 00:05:58.554842: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 21947040 exceeds 10% of free system memory.
2023-03-17 00:05:58.555021: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 22966680 exceeds 10% of free system memory.
2023-03-17 00:05:58.555035: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 22966680 exceeds 10% of free system memory.
2023-03-17 00:05:58.555350: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 23374080 exceeds 10% of free system memory.


120/120 [==============================] - 2s 5ms/step - loss: 1.1084
Epoch 2/100
120/120 [==============================] - 1s 4ms/step - loss: 1.1061
Epoch 3/100
120/120 [==============================] - 0s 4ms/step - loss: 1.1044
Epoch 4/100
120/120 [==============================] - 0s 4ms/step - loss: 1.1030
Epoch 5/100
120/120 [==============================] - 0s 4ms/step - loss: 1.1020
Epoch 6/100
120/120 [==============================] - 0s 4ms/step - loss: 1.1012
Epoch 7/100
120/120 [==============================] - 0s 4ms/step - loss: 1.1005
Epoch 8/100
120/120 [==============================] - 0s 4ms/step - loss: 1.0999
Epoch 9/100
120/120 [==============================] - 0s 4ms/step - loss: 1.0995
Epoch 10/100
120/120 [==============================] - 0s 4ms/step - loss: 1.0991
Epoch 11/100
120/120 [==============================] - 0s 4ms/step - loss: 1.0988
Epoch 12/100
120/120 [==============================] - 0s 4ms/step - loss: 1.0985
Epoch 13/100
120/120 [===

In [13]:
# model.save("Models/cdt_2")

### Evaluating

### Test

In [14]:
model.evaluate(_x_test, _y_test)

40/40 [==============================] - 0s 2ms/step - loss: 1.0997


1.0996538400650024

In [15]:
pred_test = model.predict(_x_test)
pred_test_ind = np.argmax(pred_test, axis=1)

40/40 [==============================] - 0s 3ms/step


In [16]:
print("Test correct:", np.sum(pred_test_ind == _y_test_ind))
print("Test accuracy:", np.sum(pred_test_ind == _y_test_ind)/pred_test_ind.shape[0])

Test correct: 400
Test accuracy: 0.3189792663476874


In [17]:
confusion_matrix(_y_test_ind, pred_test_ind)

array([[ 27,   0, 396],
       [ 20,   0, 403],
       [ 35,   0, 373]])

### Valitation

In [18]:
model.evaluate(_x_validation, _y_validation)

40/40 [==============================] - 0s 2ms/step - loss: 1.1093


1.1093082427978516

In [19]:
pred_validation = model.predict(_x_validation)
pred_validation_ind = np.argmax(pred_validation, axis=1)

40/40 [==============================] - 0s 3ms/step


In [20]:
print("Validation correct:", np.sum(pred_validation_ind == _y_validation_ind))
print("Validation accuracy:", np.sum(pred_validation_ind == _y_validation_ind)/pred_validation_ind.shape[0])

Validation correct: 367
Validation accuracy: 0.2926634768740032


In [21]:
confusion_matrix(_y_validation_ind, pred_validation_ind)

array([[  0,   0, 337],
       [  0,   0, 550],
       [  0,   0, 367]])

# Draft

In [22]:
a = np.broadcast_to(np.arange(10).reshape((1, 1, -1, 1)), (50, 1, 10, 2))
